In [2]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 
from typing import Any, List, Tuple, Dict, Optional, Union

import qubx
%qubxd dev

%load_ext autoreload
%autoreload 2

from qubx.data.readers import (
    CsvStorageDataReader, QuestDBConnector,
    AsPandasFrame, AsOhlcvSeries, AsQuotes, RestoreTicksFromOHLC,
    CME_FUTURES_DAILY_SESSION, STOCK_DAILY_SESSION
)
import numpy as np

 >  [dev] installing cython rebuilding hook

⠀⠀⡰⡖⠒⠒⢒⢦⠀⠀   
⠀⢠⠃⠈⢆⣀⣎⣀⣱⡀  QUBX | Quantitative Backtesting Environment 
⠀⢳⠒⠒⡞⠚⡄⠀⡰⠁         (c) 2024, ver. Dev
⠀⠀⠱⣜⣀⣀⣈⣦⠃⠀⠀⠀ 
        


# CSV Reader

<font size=+1>Let's create instance for csv data reader. It uses path to folder with csv files as argument.</font>

In [3]:
r = CsvStorageDataReader('../tests/data/csv/')

<font size=+1>Reader provides method for getting all symbols it can access in the storage</font>

In [4]:
r.get_names()

['AAPL',
 'BTCUSDT_ohlcv_M1',
 'BTCUSDT_ohlcv_M1_sec',
 'ETHUSDT.trades',
 'quotes',
 'SPY',
 'SPY1']

<font size=+1>We can read data without any transformation - it will return records as plain list </font>

In [18]:
r.read('SPY')

[array([datetime.date(2000, 1, 3), 107.404545309841, 107.404545309841, 104.23493393897722, 105.36693799999999, 8164300.0], dtype=object),
 array([datetime.date(2000, 1, 4), 103.98585949558047, 104.37077420169946, 101.1671802569384, 101.246443, 8089800.0], dtype=object),
 array([datetime.date(2000, 1, 5), 101.38228283794643, 102.53689364411323, 99.43523586964287, 101.427563, 12177900.0], dtype=object),
 array([datetime.date(2000, 1, 6), 101.15588287295826, 102.51428774591652, 99.797478, 99.797478, 6227200.0], dtype=object),
 array([datetime.date(2000, 1, 7), 101.65396389794167, 105.59333799999999, 101.47284324957117, 105.59333799999999, 8066500.0], dtype=object),
 array([datetime.date(2000, 1, 10), 105.95558, 106.43098837892867, 105.07258350713381, 105.95558, 5741700.0], dtype=object),
 array([datetime.date(2000, 1, 11), 105.63861840614187, 105.84234580983004, 103.9632524048443, 104.687735, 7503700.0], dtype=object),
 array([datetime.date(2000, 1, 12), 104.75562164474871, 104.7556216447

<font size=+1>Select range of data we are interested in </font>

In [7]:
r.read('BTCUSDT_ohlcv_M1', '2024-01-10 15:00', '2024-01-10 15:10')

[array([Timestamp('2024-01-10 15:00:00'), 44890.2, 44895.9, 44808.3, 44812.1, 362.186], dtype=object),
 array([Timestamp('2024-01-10 15:01:00'), 44812.1, 44918.8, 44803.5, 44900.0, 932.329], dtype=object),
 array([Timestamp('2024-01-10 15:02:00'), 44900.0, 44984.0, 44875.6, 44970.3, 356.052], dtype=object),
 array([Timestamp('2024-01-10 15:03:00'), 44970.4, 45034.0, 44948.7, 45025.9, 753.383], dtype=object),
 array([Timestamp('2024-01-10 15:04:00'), 45025.9, 45026.0, 44954.6, 44992.8, 366.763], dtype=object),
 array([Timestamp('2024-01-10 15:05:00'), 44992.8, 45023.7, 44954.6, 45020.0, 292.081], dtype=object),
 array([Timestamp('2024-01-10 15:06:00'), 45020.0, 45140.0, 45010.5, 45123.5, 1130.425], dtype=object),
 array([Timestamp('2024-01-10 15:07:00'), 45123.4, 45169.5, 45085.5, 45148.5, 1111.855], dtype=object),
 array([Timestamp('2024-01-10 15:08:00'), 45148.5, 45162.0, 45113.4, 45162.0, 313.443], dtype=object),
 array([Timestamp('2024-01-10 15:09:00'), 45161.9, 45191.4, 45137.2, 45

<font size=+1>We can read it by chunks if batch streaming processing</font>

In [8]:
# let's read by 100 records
for x in r.read('BTCUSDT_ohlcv_M1', '2024-01-10', '2024-01-11', chunksize=100):
    print(f"{x[0][0]} ~ {x[-1][0]} >> length is {len(x)} records")

2024-01-10 00:00:00 ~ 2024-01-10 01:39:00 >> length is 100 records
2024-01-10 01:40:00 ~ 2024-01-10 03:19:00 >> length is 100 records
2024-01-10 03:20:00 ~ 2024-01-10 04:59:00 >> length is 100 records
2024-01-10 05:00:00 ~ 2024-01-10 06:39:00 >> length is 100 records
2024-01-10 06:40:00 ~ 2024-01-10 08:19:00 >> length is 100 records
2024-01-10 08:20:00 ~ 2024-01-10 09:59:00 >> length is 100 records
2024-01-10 10:00:00 ~ 2024-01-10 11:39:00 >> length is 100 records
2024-01-10 11:40:00 ~ 2024-01-10 13:19:00 >> length is 100 records
2024-01-10 13:20:00 ~ 2024-01-10 14:59:00 >> length is 100 records
2024-01-10 15:00:00 ~ 2024-01-10 16:39:00 >> length is 100 records
2024-01-10 16:40:00 ~ 2024-01-10 18:19:00 >> length is 100 records
2024-01-10 18:20:00 ~ 2024-01-10 19:59:00 >> length is 100 records
2024-01-10 20:00:00 ~ 2024-01-10 21:39:00 >> length is 100 records
2024-01-10 21:40:00 ~ 2024-01-10 23:19:00 >> length is 100 records
2024-01-10 23:20:00 ~ 2024-01-11 00:00:00 >> length is 41 reco

<font size=+1>We can apply data transformation on this data. Let's say we want pandas dataframe for some further analysis:</font>

In [19]:
r.read('SPY',  transform=AsPandasFrame())

,close,high,low,open,volume
Date,,,,,
2000-01-03,105.366938,107.404545,104.234934,107.404545,8164300.0
2000-01-04,101.246443,104.370774,101.167180,103.985859,8089800.0
2000-01-05,101.427563,102.536894,99.435236,101.382283,12177900.0
2000-01-06,99.797478,102.514288,99.797478,101.155883,6227200.0
2000-01-07,105.593338,105.593338,101.472843,101.653964,8066500.0
...,...,...,...,...,...
2017-04-17,234.570007,234.570007,232.880005,233.110001,63559500.0
2017-04-18,233.869995,234.490005,233.080002,233.720001,80188300.0
2017-04-19,233.440002,234.949997,233.179993,234.520004,66861500.0


<font size=+1>We can specify custom timestamps formats if need </font>

In [20]:
r.read('AAPL',  transform=AsPandasFrame(), timestamp_formatters=["%d-%m-%Y"])

,Close,High,Low,Open,Volume
Date,,,,,
2016-06-01,98.46,99.54,98.33,99.02,29173285
2016-06-02,97.72,97.84,96.63,97.60,40191600
2016-06-03,97.92,98.27,97.45,97.79,28504888
2016-06-06,98.63,101.89,97.55,97.99,23292504
2016-06-07,99.03,99.87,98.96,99.25,22409450
...,...,...,...,...,...
2016-11-28,111.57,112.46,111.39,111.43,27193983
2016-11-29,111.46,112.03,110.07,110.78,28528750
2016-11-30,110.52,112.20,110.27,111.60,36162258


<font size=+1>Here we can read quotes and transform them to dataframe</font>

In [33]:
r.read('quotes', transform=AsPandasFrame())

,ask,askvol,bid,bidvol
time,,,,
2017-08-24 13:01:12,10.02,200,10.00,100
2017-08-24 13:01:13,10.00,20000,9.98,100
2017-08-24 13:01:14,10.03,200,10.00,100
2017-08-24 13:01:15,10.02,20000,10.01,100
2017-08-24 13:01:16,10.01,200,9.98,100
...,...,...,...,...
2017-08-24 13:09:27,9.33,200,9.30,10000
2017-08-24 13:09:28,9.33,200,9.31,10000
2017-08-24 13:09:29,9.35,200,9.33,100


<font size=+1>It's possible to transform data into Qubx OHLCV series</font>

In [15]:
r.read('BTCUSDT_ohlcv_M1', transform=AsOhlcvSeries())

                        open     high      low    close   volume  \
timestamp                                                          
2024-01-01 00:00:00  42314.0  42335.8  42289.6  42331.9  289.641   
2024-01-01 00:01:00  42331.9  42353.1  42331.8  42350.4  202.444   
2024-01-01 00:02:00  42350.4  42370.8  42349.6  42360.2  271.521   
2024-01-01 00:03:00  42360.1  42405.8  42360.1  42405.8  392.238   
2024-01-01 00:04:00  42405.7  42437.2  42405.7  42437.1  568.366   
...                      ...      ...      ...      ...      ...   
2024-02-15 06:13:00  51956.3  51977.9  51956.3  51977.2   86.380   
2024-02-15 06:14:00  51977.8  51984.9  51961.5  51983.3  271.145   
2024-02-15 06:15:00  51983.3  52015.0  51983.2  51995.9  191.007   
2024-02-15 06:16:00  51996.0  52014.7  51965.9  51966.0  261.885   
2024-02-15 06:17:00  51965.9  52004.7  51950.0  51950.4  223.358   

                     bought_volume  
timestamp                           
2024-01-01 00:00:00            0.0  
2024

<font size=+1>Resample 1min bars data to daily timeframe on the fly</font>

In [35]:
r.read('BTCUSDT_ohlcv_M1', transform=AsOhlcvSeries('1d'))

               open     high      low    close      volume  bought_volume
timestamp                                                                
2024-01-01  42314.0  44266.0  42207.9  44230.2  206424.144            0.0
2024-01-02  44230.3  45950.0  44200.9  44979.8  459798.523            0.0
2024-01-03  44979.7  45582.3  40333.0  42849.5  595855.225            0.0
2024-01-04  42849.5  44840.8  42625.0  44143.8  333923.098            0.0
2024-01-05  44143.8  44500.0  42300.0  44145.4  374967.791            0.0
2024-01-06  44145.3  44214.6  43391.3  43956.7  138542.797            0.0
2024-01-07  43956.6  44486.9  43557.5  43916.9  180710.714            0.0
2024-01-08  43916.9  47312.0  43158.1  46972.7  495087.983            0.0
2024-01-09  46972.6  48100.0  44417.2  46114.0  513362.773            0.0
2024-01-10  46113.9  47699.3  44291.5  46662.9  668657.474            0.0
2024-01-11  46663.0  49027.5  45600.0  46337.8  596698.379            0.0
2024-01-12  46337.9  46536.2  41370.0 

<font size=+1>Let's make OHLC from quotes and make 2 Min bars out of them: </font>

In [16]:
qbx_ohlc = r.read('quotes', transform=AsOhlcvSeries('2Min'))
qbx_ohlc

                       open    high    low   close  volume  bought_volume
timestamp                                                                
2017-08-24 13:00:00  10.010  10.050  9.810  10.050     0.0            0.0
2017-08-24 13:02:00  10.030  10.095  9.565   9.685     0.0            0.0
2017-08-24 13:04:00   9.710   9.925  9.665   9.915     0.0            0.0
2017-08-24 13:06:00   9.915   9.920  9.530   9.530     0.0            0.0
2017-08-24 13:08:00   9.560   9.585  9.240   9.375     0.0            0.0

In [17]:
r.read('quotes', '2017-08-24 13:09:29', '2017-08-24 13:09:29')

[array([Timestamp('2017-08-24 13:09:29'), 9.35, 9.33, 200, 100], dtype=object)]

<font size=+1>Qubx series can be converted to pandas</font>

In [231]:
qbx_ohlc.pd()

,open,high,low,close,volume,bought_volume
timestamp,,,,,,
2017-08-24 13:00:00,10.010,10.050,9.810,10.050,0.0,0.0
2017-08-24 13:02:00,10.030,10.095,9.565,9.685,0.0,0.0
2017-08-24 13:04:00,9.710,9.925,9.665,9.915,0.0,0.0
2017-08-24 13:06:00,9.915,9.920,9.530,9.530,0.0,0.0
2017-08-24 13:08:00,9.560,9.585,9.240,9.375,0.0,0.0


<font size=+1>Create OHLC from trades</font>

In [275]:
# that's trades from Binance - timestamps are msec
r.read('ETHUSDT.trades', transform=AsOhlcvSeries('5s', timestamp_units='ms'))

                        open     high      low    close        volume  \
timestamp                                                               
2024-04-20 00:00:00  3054.91  3054.92  3054.20  3054.68  5.939874e+05   
2024-04-20 00:00:05  3054.59  3055.32  3054.54  3055.31  4.855402e+05   
2024-04-20 00:00:10  3055.32  3056.15  3054.57  3054.91  1.186494e+06   
2024-04-20 00:00:15  3054.90  3056.45  3054.90  3056.31  3.506018e+05   
2024-04-20 00:00:20  3056.30  3056.30  3055.10  3055.53  2.049955e+05   
2024-04-20 00:00:25  3055.53  3057.21  3055.53  3056.52  9.013015e+05   

                     bought_volume  
timestamp                           
2024-04-20 00:00:00   370283.35706  
2024-04-20 00:00:05    71978.41917  
2024-04-20 00:00:10   282420.97724  
2024-04-20 00:00:15    15416.16702  
2024-04-20 00:00:20   127399.24401  
2024-04-20 00:00:25   417706.04275  

<font size=+1>When need just plain list of quotes</font>

In [286]:
r.read('quotes', transform=AsQuotes())

[[2017-08-24T13:01:12.000000000]	10.00000 (100.0) | 10.02000 (200.0),
 [2017-08-24T13:01:13.000000000]	9.98000 (100.0) | 10.00000 (20000.0),
 [2017-08-24T13:01:14.000000000]	10.00000 (100.0) | 10.03000 (200.0),
 [2017-08-24T13:01:15.000000000]	10.01000 (100.0) | 10.02000 (20000.0),
 [2017-08-24T13:01:16.000000000]	9.98000 (100.0) | 10.01000 (200.0),
 [2017-08-24T13:01:17.000000000]	9.96000 (100.0) | 9.99000 (200.0),
 [2017-08-24T13:01:18.000000000]	9.96000 (10000.0) | 9.99000 (200.0),
 [2017-08-24T13:01:19.000000000]	9.97000 (100.0) | 9.99000 (200.0),
 [2017-08-24T13:01:20.000000000]	9.95000 (10000.0) | 9.98000 (20000.0),
 [2017-08-24T13:01:21.000000000]	9.97000 (10000.0) | 9.98000 (200.0),
 [2017-08-24T13:01:22.000000000]	9.94000 (100.0) | 9.95000 (20000.0),
 [2017-08-24T13:01:23.000000000]	9.96000 (100.0) | 9.99000 (200.0),
 [2017-08-24T13:01:24.000000000]	9.94000 (100.0) | 9.96000 (200.0),
 [2017-08-24T13:01:25.000000000]	9.93000 (10000.0) | 9.96000 (200.0),
 [2017-08-24T13:01:26.00

<font size=+1>Let's <i>'restore'</i> quotes from OHLC bars ;) </font>

In [293]:
r.read('BTCUSDT_ohlcv_M1', '2024-01-01 10:00', '2024-01-01 10:05', 
       transform=RestoreTicksFromOHLC())

[[2024-01-01T10:00:06.000000000]	42688.80000 (1000000000.0) | 42688.80000 (1000000000.0),
 [2024-01-01T10:00:24.000000000]	42686.20000 (1000000000.0) | 42686.20000 (1000000000.0),
 [2024-01-01T10:00:36.000000000]	42696.30000 (1000000000.0) | 42696.30000 (1000000000.0),
 [2024-01-01T10:00:54.000000000]	42695.40000 (1000000000.0) | 42695.40000 (1000000000.0),
 [2024-01-01T10:01:06.000000000]	42695.40000 (1000000000.0) | 42695.40000 (1000000000.0),
 [2024-01-01T10:01:24.000000000]	42687.80000 (1000000000.0) | 42687.80000 (1000000000.0),
 [2024-01-01T10:01:36.000000000]	42698.90000 (1000000000.0) | 42698.90000 (1000000000.0),
 [2024-01-01T10:01:54.000000000]	42696.60000 (1000000000.0) | 42696.60000 (1000000000.0),
 [2024-01-01T10:02:06.000000000]	42696.60000 (1000000000.0) | 42696.60000 (1000000000.0),
 [2024-01-01T10:02:24.000000000]	42706.80000 (1000000000.0) | 42706.80000 (1000000000.0),
 [2024-01-01T10:02:36.000000000]	42692.00000 (1000000000.0) | 42692.00000 (1000000000.0),
 [2024-01-

<font size=+1>We can also emulate trades from ohlc bar volumes </font>

In [301]:
r.read('BTCUSDT_ohlcv_M1', '2024-01-01 10:00', '2024-01-01 10:05', 
       transform=RestoreTicksFromOHLC(trades=True))

[[2024-01-01T10:00:06.000000000]	42688.80000 (1000000000.0) | 42688.80000 (1000000000.0),
 [2024-01-01T10:00:06.000000000]	42688.80000 (140.6) <???> ,
 [2024-01-01T10:00:24.000000000]	42686.20000 (1000000000.0) | 42686.20000 (1000000000.0),
 [2024-01-01T10:00:24.000000000]	42686.20000 (357.0) <???> ,
 [2024-01-01T10:00:36.000000000]	42696.30000 (1000000000.0) | 42696.30000 (1000000000.0),
 [2024-01-01T10:00:36.000000000]	42696.30000 (48.7) <???> ,
 [2024-01-01T10:00:54.000000000]	42695.40000 (1000000000.0) | 42695.40000 (1000000000.0),
 [2024-01-01T10:01:06.000000000]	42695.40000 (1000000000.0) | 42695.40000 (1000000000.0),
 [2024-01-01T10:01:06.000000000]	42695.40000 (458.8) <???> ,
 [2024-01-01T10:01:24.000000000]	42687.80000 (1000000000.0) | 42687.80000 (1000000000.0),
 [2024-01-01T10:01:24.000000000]	42687.80000 (72.4) <???> ,
 [2024-01-01T10:01:36.000000000]	42698.90000 (1000000000.0) | 42698.90000 (1000000000.0),
 [2024-01-01T10:01:36.000000000]	42698.90000 (138.8) <???> ,
 [2024

<font size=+1>In case if we need emulate some session times</font>

For example stock market opens at 9:30 so we need to see first tick at this time 

In [308]:
r.read('SPY', transform=RestoreTicksFromOHLC(
    daily_session_start_end=STOCK_DAILY_SESSION
))[:4] # type: ignore

[[2000-01-03T09:30:00.100000000]	107.40455 (1000000000.0) | 107.40455 (1000000000.0),
 [2000-01-03T11:00:00.000000000]	107.40455 (1000000000.0) | 107.40455 (1000000000.0),
 [2000-01-03T13:00:00.000000000]	104.23493 (1000000000.0) | 104.23493 (1000000000.0),
 [2000-01-03T15:59:59.900000000]	105.36694 (1000000000.0) | 105.36694 (1000000000.0)]

# Quest DB

We can do the same things with other connector - for example QuestDB 

In [23]:
qreader = QuestDBConnector('user=admin password=quest host=localhost port=8812')

2024-05-08 20:03:27.392 [ ℹ️ ] Connecting to QuestDB ...


In [24]:
qreader.read("BINANCEF.ETHUSDT", '2023-01-01', '2023-01-02', timeframe='5m', transform=AsPandasFrame())

,close,count,high,low,open,quote_volume,taker_buy_quote_volume,taker_buy_volume,volume
timestamp,,,,,,,,,
2023-01-01 00:00:00,1196.29,1202.0,1196.30,1195.42,1196.02,3.189650e+06,1.656171e+06,1384.950,2667.388
2023-01-01 00:05:00,1195.49,888.0,1196.68,1195.49,1196.51,2.033846e+06,6.262188e+05,523.413,1700.149
2023-01-01 00:10:00,1194.25,2060.0,1195.48,1194.00,1195.45,6.401944e+06,3.201785e+06,2680.286,5359.232
2023-01-01 00:15:00,1196.00,1274.0,1196.12,1194.65,1194.71,3.122414e+06,2.307215e+06,1930.066,2611.958
2023-01-01 00:20:00,1195.96,842.0,1196.46,1195.94,1196.32,1.571853e+06,6.017303e+05,502.994,1313.989
...,...,...,...,...,...,...,...,...,...
2023-01-01 23:35:00,1200.12,1188.0,1200.47,1198.82,1199.12,2.533064e+06,1.547148e+06,1289.895,2111.763
2023-01-01 23:40:00,1200.05,905.0,1200.90,1199.94,1200.15,1.945826e+06,9.407156e+05,783.523,1620.790
2023-01-01 23:45:00,1199.34,837.0,1199.52,1199.03,1199.51,1.427222e+06,6.588002e+05,549.352,1190.088


In [25]:
qreader.read("BINANCEF.ETHUSDT", '2023-01-01', '2023-01-02', timeframe='5m', transform=AsOhlcvSeries())

                        open     high      low    close        volume  \
timestamp                                                               
2023-01-01 00:00:00  1196.02  1196.30  1195.42  1196.29  3.189650e+06   
2023-01-01 00:05:00  1196.51  1196.68  1195.49  1195.49  2.033846e+06   
2023-01-01 00:10:00  1195.45  1195.48  1194.00  1194.25  6.401944e+06   
2023-01-01 00:15:00  1194.71  1196.12  1194.65  1196.00  3.122414e+06   
2023-01-01 00:20:00  1196.32  1196.46  1195.94  1195.96  1.571853e+06   
...                      ...      ...      ...      ...           ...   
2023-01-01 23:35:00  1199.12  1200.47  1198.82  1200.12  2.533064e+06   
2023-01-01 23:40:00  1200.15  1200.90  1199.94  1200.05  1.945826e+06   
2023-01-01 23:45:00  1199.51  1199.52  1199.03  1199.34  1.427222e+06   
2023-01-01 23:50:00  1199.91  1200.16  1199.06  1200.16  2.832485e+06   
2023-01-01 23:55:00  1200.68  1200.69  1199.75  1199.96  2.291290e+06   

                     bought_volume  
timestamp    

In [26]:
qreader.read("BINANCEF.ETHUSDT", '2023-01-01', '2023-01-02', timeframe='5m', transform=RestoreTicksFromOHLC(trades=True))

[[2023-01-01T00:00:30.000000000]	1196.02000 (1000000000.0) | 1196.02000 (1000000000.0),
 [2023-01-01T00:00:30.000000000]	1196.02000 (1600.4) <???> ,
 [2023-01-01T00:02:00.000000000]	1195.42000 (1000000000.0) | 1195.42000 (1000000000.0),
 [2023-01-01T00:02:00.000000000]	1195.42000 (720.2) <???> ,
 [2023-01-01T00:03:00.000000000]	1196.30000 (1000000000.0) | 1196.30000 (1000000000.0),
 [2023-01-01T00:03:00.000000000]	1196.30000 (26.7) <???> ,
 [2023-01-01T00:04:30.000000000]	1196.29000 (1000000000.0) | 1196.29000 (1000000000.0),
 [2023-01-01T00:05:30.000000000]	1196.51000 (1000000000.0) | 1196.51000 (1000000000.0),
 [2023-01-01T00:05:30.000000000]	1196.51000 (289.0) <???> ,
 [2023-01-01T00:07:00.000000000]	1196.68000 (1000000000.0) | 1196.68000 (1000000000.0),
 [2023-01-01T00:07:00.000000000]	1196.68000 (1734.2) <???> ,
 [2023-01-01T00:08:00.000000000]	1195.49000 (1000000000.0) | 1195.49000 (1000000000.0),
 [2023-01-01T00:08:00.000000000]	1195.49000 (0.0) <???> ,
 [2023-01-01T00:09:30.000

We can use deltas instead of exact times

In [27]:
qreader.read("BINANCEF.ETHUSDT", '2023-01-01', '1d', timeframe='5m', transform=AsPandasFrame())

,close,count,high,low,open,quote_volume,taker_buy_quote_volume,taker_buy_volume,volume
timestamp,,,,,,,,,
2023-01-01 00:00:00,1196.29,1202.0,1196.30,1195.42,1196.02,3.189650e+06,1.656171e+06,1384.950,2667.388
2023-01-01 00:05:00,1195.49,888.0,1196.68,1195.49,1196.51,2.033846e+06,6.262188e+05,523.413,1700.149
2023-01-01 00:10:00,1194.25,2060.0,1195.48,1194.00,1195.45,6.401944e+06,3.201785e+06,2680.286,5359.232
2023-01-01 00:15:00,1196.00,1274.0,1196.12,1194.65,1194.71,3.122414e+06,2.307215e+06,1930.066,2611.958
2023-01-01 00:20:00,1195.96,842.0,1196.46,1195.94,1196.32,1.571853e+06,6.017303e+05,502.994,1313.989
...,...,...,...,...,...,...,...,...,...
2023-01-01 23:35:00,1200.12,1188.0,1200.47,1198.82,1199.12,2.533064e+06,1.547148e+06,1289.895,2111.763
2023-01-01 23:40:00,1200.05,905.0,1200.90,1199.94,1200.15,1.945826e+06,9.407156e+05,783.523,1620.790
2023-01-01 23:45:00,1199.34,837.0,1199.52,1199.03,1199.51,1.427222e+06,6.588002e+05,549.352,1190.088


In [28]:
qreader.read("BINANCEF.ETHUSDT", '2023-01-01', '-1d', timeframe='5m', transform=AsPandasFrame())

,close,count,high,low,open,quote_volume,taker_buy_quote_volume,taker_buy_volume,volume
timestamp,,,,,,,,,
2022-12-31 00:00:00,1198.75,1397.0,1199.66,1198.69,1199.57,3.072898e+06,1.430514e+06,1193.120,2562.743
2022-12-31 00:05:00,1198.07,952.0,1198.64,1198.04,1198.14,1.788619e+06,1.150721e+06,960.284,1492.669
2022-12-31 00:10:00,1196.80,2974.0,1197.97,1196.46,1197.84,7.588446e+06,3.121122e+06,2607.008,6338.849
2022-12-31 00:15:00,1197.42,1015.0,1197.60,1196.87,1196.99,2.516592e+06,1.705503e+06,1424.535,2101.995
2022-12-31 00:20:00,1198.33,1107.0,1198.50,1198.00,1198.00,2.204805e+06,1.021694e+06,852.629,1839.940
...,...,...,...,...,...,...,...,...,...
2022-12-31 23:35:00,1196.55,1894.0,1197.48,1196.55,1197.43,4.805083e+06,2.006399e+06,1676.145,4014.308
2022-12-31 23:40:00,1196.78,639.0,1197.00,1196.54,1196.56,1.419789e+06,7.113449e+05,594.393,1186.410
2022-12-31 23:45:00,1196.32,767.0,1196.81,1196.14,1196.80,1.960512e+06,7.446607e+05,622.429,1638.606


In [30]:
qreader.read("BINANCEF.ETHUSDT", '1d', '2023-01-01', timeframe='5m', transform=AsPandasFrame())

,close,count,high,low,open,quote_volume,taker_buy_quote_volume,taker_buy_volume,volume
timestamp,,,,,,,,,
2022-12-31 00:00:00,1198.75,1397.0,1199.66,1198.69,1199.57,3.072898e+06,1.430514e+06,1193.120,2562.743
2022-12-31 00:05:00,1198.07,952.0,1198.64,1198.04,1198.14,1.788619e+06,1.150721e+06,960.284,1492.669
2022-12-31 00:10:00,1196.80,2974.0,1197.97,1196.46,1197.84,7.588446e+06,3.121122e+06,2607.008,6338.849
2022-12-31 00:15:00,1197.42,1015.0,1197.60,1196.87,1196.99,2.516592e+06,1.705503e+06,1424.535,2101.995
2022-12-31 00:20:00,1198.33,1107.0,1198.50,1198.00,1198.00,2.204805e+06,1.021694e+06,852.629,1839.940
...,...,...,...,...,...,...,...,...,...
2022-12-31 23:35:00,1196.55,1894.0,1197.48,1196.55,1197.43,4.805083e+06,2.006399e+06,1676.145,4014.308
2022-12-31 23:40:00,1196.78,639.0,1197.00,1196.54,1196.56,1.419789e+06,7.113449e+05,594.393,1186.410
2022-12-31 23:45:00,1196.32,767.0,1196.81,1196.14,1196.80,1.960512e+06,7.446607e+05,622.429,1638.606
